In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from skimage.morphology import skeletonize
from skimage.util import invert
import random




In [1]:
import cv2
import numpy as np
import random
from skimage.morphology import skeletonize

def crack_detection(img):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply a blur to the grayscale image
    blurred_img = cv2.GaussianBlur(gray_image, (25, 25), 0)

    # Thresholding to get a binary image (inverse binary to highlight cracks)
    _, binary_img = cv2.threshold(blurred_img, 127, 255, cv2.THRESH_BINARY_INV)

    # Skeletonize the inverted binary image to get a thin representation of the cracks
    skeleton = skeletonize(binary_img // 255).astype(np.uint8) * 255  # Convert boolean to uint8 for OpenCV

    # Detect edges using Canny edge detection
    med_val = np.median(gray_image)
    lower = int(max(0, 0.7 * med_val))
    upper = int(min(255, 1.3 * med_val))
    edges = cv2.Canny(image=gray_image, threshold1=lower, threshold2=upper + 50)

    # Threshold edges for binary image
    _, edges = cv2.threshold(edges, 127, 255, cv2.THRESH_BINARY)

    # Get the coordinates of the skeleton points
    skeleton_points = np.column_stack(np.where(skeleton > 0))

    if len(skeleton_points) == 0:
        return img, []  # If no skeleton points are detected, return original image and an empty list

    # Set the number of perpendicular lines to visualize
    num_perpendicular_lines = 20
    perpendicular_distances = []  # To store the distances of intersections

    # Draw detected edges in red color on the original image
    img_with_edges = img.copy()
    img_with_edges[edges > 0] = [0, 0, 255]  # Red color for edges

    # Iterate to add perpendicular lines
    for _ in range(num_perpendicular_lines):
        # Randomly select a point on the skeleton
        random_point = skeleton_points[random.randint(0, len(skeleton_points) - 1)]

        # Get the neighbors for directional vector (choosing nearby points)
        index = np.where((skeleton_points[:, 0] == random_point[0]) & (skeleton_points[:, 1] == random_point[1]))[0][0]
        prev_point = skeleton_points[index - 1] if index > 0 else random_point
        next_point = skeleton_points[index + 1] if index < len(skeleton_points) - 1 else random_point

        # Calculate the direction vector and perpendicular vector
        direction_vector = (next_point[0] - prev_point[0], next_point[1] - prev_point[1])
        perpendicular_vector = (-direction_vector[1], direction_vector[0])

        # Normalize the perpendicular vector
        length = np.sqrt(perpendicular_vector[0] ** 2 + perpendicular_vector[1] ** 2)
        if length == 0:  # Avoid division by zero
            continue
        perpendicular_vector = (perpendicular_vector[0] / length, perpendicular_vector[1] / length)

        # Define the length of the perpendicular line
        perpendicular_length = 30

        # Calculate the start and end points of the perpendicular line
        start_perpendicular = (int(random_point[0] - perpendicular_length * perpendicular_vector[0]),
                               int(random_point[1] - perpendicular_length * perpendicular_vector[1]))

        end_perpendicular = (int(random_point[0] + perpendicular_length * perpendicular_vector[0]),
                             int(random_point[1] + perpendicular_length * perpendicular_vector[1]))

        # Draw the perpendicular line on the image with edges
        cv2.line(img_with_edges, (start_perpendicular[1], start_perpendicular[0]),
                 (end_perpendicular[1], end_perpendicular[0]), (0, 255, 0), 1)

        # Detect intersections of the perpendicular line with edges
        intersections = []
        rr, cc = np.linspace(start_perpendicular[0], end_perpendicular[0], num=100).astype(int), \
                 np.linspace(start_perpendicular[1], end_perpendicular[1], num=100).astype(int)

        for r, c in zip(rr, cc):
            if 0 <= r < edges.shape[0] and 0 <= c < edges.shape[1]:
                if edges[r, c] > 0:  # Check if the pixel is part of the edge
                    intersections.append((r, c))

        # If there are intersections, plot them and calculate distance
        if len(intersections) >= 2:
            first_intersection = intersections[0]
            last_intersection = intersections[-1]

            # Plot intersection points
            cv2.circle(img_with_edges, (first_intersection[1], first_intersection[0]), 3, (255, 0, 0), -1)
            cv2.circle(img_with_edges, (last_intersection[1], last_intersection[0]), 3, (255, 0, 0), -1)

            # Calculate the distance between intersections
            distance = np.sqrt((last_intersection[0] - first_intersection[0]) ** 2 +
                               (last_intersection[1] - first_intersection[1]) ** 2)
            perpendicular_distances.append(distance)
        else:
            perpendicular_distances.append(0)  # No intersections found, add zero

    # Return the modified image and the calculated distances (for further use)
    return img_with_edges, perpendicular_distances

In [ ]:
print(cv2.useOptimized())

# Open the video file or capture from webcam
video_capture = cv2.VideoCapture(0)  # You can also use 0 to capture from webcam

# Get the frame width, height, and frame rate of the original video
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(video_capture.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the processed video
output_video = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (frame_width, frame_height), isColor=False)

while True:
    # Read a frame from the video
    ret, frame = video_capture.read()

    # If the frame was not successfully read (end of video), break the loop
    if not ret:
        break
    img_with_edges , perpendicular_distances = crack_detection(frame)
    # Convert the frame to grayscale
    # gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # # Write the processed frame into the output video

    # # Optional: Display the frame being processed (for debugging or visual feedback)
    cv2.imshow('Grayscale Video', img_with_edges)
    print(perpendicular_distances)

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects and close any OpenCV windows
video_capture.release()
output_video.release()
cv2.destroyAllWindows()

# Get The model


In [6]:
from ultralytics import YOLO
import cv2
import os
import time  # Added to manage the time intervals

print(cv2.useOptimized())

# Load your trained YOLOv8 segmentation model
model = YOLO("Model/train5/weights/best.pt")  # path to your best weights

# Capture video from webcam
cap = cv2.VideoCapture(0)



# Create a directory to store cropped crack images
output_folder = 'New_Last_Cracks'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

frame_count = 0  # To uniquely name each saved cropped crack
last_save_time = time.time()  # Record the time of the last saved image

save_interval = 1  # Save a new photo every 5 seconds

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform crack detection with segmentation
    results = model(frame)

    # Extract detection results
    for result in results:
        # Loop through each detection
        for box in result.boxes:
            # Get bounding box coordinates
            xmin, ymin, xmax, ymax = map(int, box.xyxy[0])  # xyxy are the bounding box coordinates

            # Calculate width and height of the bounding box
            width = xmax - xmin
            height = ymax - ymin

            # Check if both width and height are greater than or equal to 300
            if width >= 300 or height >= 300:
                # Get the current time
                current_time = time.time()

                # Check if the 5 seconds have passed since the last saved image
                if current_time - last_save_time >= save_interval:
                    # Crop the crack from the frame
                    cropped_crack = frame[ymin+10:ymax-10, xmin+10:xmax-10]

                    # Generate a unique name for the crack image
                    crack_image_name = f'crack_{frame_count}.png'
                    crack_image_path = os.path.join(output_folder, crack_image_name)

                    # Save the cropped crack image
                    # cv2.imwrite(crack_image_path, cropped_crack)
                    frame_count += 1  # Increment for the next crack

                    # Update the last save time
                    last_save_time = current_time
                    result_frame = results[0].plot()

                    result_frame , perpendicular_distances = crack_detection(cropped_crack)


                    if perpendicular_distances:
                        cv2.imshow('Real-time Crack Detection', result_frame)
                    else:
                        cv2.imshow('Real-time Crack Detection', cropped_crack)  # Show cropped frame if no distances found

    # Display the results with segmentation masks
    cv2.imshow('normal camera', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

True

0: 384x640 1 cracks, 55.4ms
Speed: 1.8ms preprocess, 55.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.4ms
Speed: 1.8ms preprocess, 54.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.7ms
Speed: 1.5ms preprocess, 40.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 31.6ms
Speed: 1.1ms preprocess, 31.6ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.8ms
Speed: 1.1ms preprocess, 29.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.9ms
Speed: 1.1ms preprocess, 29.9ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 34.5ms
Speed: 1.6ms preprocess, 34.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.0ms
Speed: 1.0ms preprocess, 36.0ms inf

KeyboardInterrupt: 